In [1]:
import numpy as np
from collections import deque, defaultdict
from copy import deepcopy

In [2]:
import sys
sys.setrecursionlimit(10000)

In [3]:
test = ['#.#####################',
        '#.......#########...###',
        '#######.#########.#.###',
        '###.....#.>.>.###.#.###',
        '###v#####.#v#.###.#.###',
        '###.>...#.#.#.....#...#',
        '###v###.#.#.#########.#',
        '###...#.#.#.......#...#',
        '#####.#.#.#######.#.###',
        '#.....#.#.#.......#...#',
        '#.#####.#.#.#########v#',
        '#.#...#...#...###...>.#',
        '#.#.#v#######v###.###v#',
        '#...#.>.#...>.>.#.###.#',
        '#####v#.#.###v#.#.###.#',
        '#.....#...#...#.#.#...#',
        '#.#########.###.#.#.###',
        '#...###...#...#...#.###',
        '###.###.#.###v#####v###',
        '#...#...#.#.>.>.#.>.###',
        '#.###.###.#.###.#.#v###',
        '#.....###...###...#...#',
        '#####################.#']

data = np.genfromtxt('day23_input.txt', dtype=str, delimiter='\n', comments=None)

In [4]:
def gen_graph(data):
    graph = []
    for line in data:
        graph.append(list(line))
    return np.array(graph)

def good_pos(graph, pos, xy, part):
    if pos[0]+xy[0] < 0 or pos[0]+xy[0] >= len(graph):
        return False
    elif pos[1]+xy[1] < 0 or pos[1]+xy[1] >= len(graph[pos[0]+xy[0]]):
        return False
    
    if graph[pos[0]+xy[0], pos[1]+xy[1]] == '#':
        return False
    
    if part == 1:
        if graph[pos[0]+xy[0], pos[1]+xy[1]] == '^' and xy[0] == 1:
            return False
        elif graph[pos[0]+xy[0], pos[1]+xy[1]] == 'v' and xy[0] == -1:
            return False
        elif graph[pos[0]+xy[0], pos[1]+xy[1]] == '<' and xy[1] == 1:
            return False
        elif graph[pos[0]+xy[0], pos[1]+xy[1]] == '>' and xy[1] == -1:
            return False
    
    return True

def DFS(graph, pos, path, end, distance, part=1):
    path.append(pos)
    
    if pos[0] == end[0] and pos[1] == end[1]:
        #print(path, distance)
        if len(path) > distance:
            return int(len(path))
        else:
            return distance
    
    dxdy = [[-1,0], [1,0], [0,-1], [0,1]]
    
    for xy in dxdy:
        if part == 1 and\
           ((graph[pos[0],pos[1]] == '^' and xy[0] != -1) or\
           (graph[pos[0],pos[1]] == 'v' and xy[0] != 1) or\
           (graph[pos[0],pos[1]] == '<' and xy[1] != -1) or\
           (graph[pos[0],pos[1]] == '>' and xy[1] != 1)):
            continue
        if good_pos(graph, pos, xy, part) and (pos[0]+xy[0],pos[1]+xy[1]) not in path:
            distance = DFS(graph, (pos[0]+xy[0],pos[1]+xy[1]), deepcopy(path), end, distance, part)
            
    return distance
            
def part1(data):
    graph = gen_graph(data)
    start = (0, 1)
    end = (len(graph)-1, len(graph[0])-2)
    distance = DFS(graph, start, [], end, 0)
    return distance-1

In [5]:
print(part1(test))
print('Part 1 result:', part1(data))

94
Part 1 result: 2298


In [6]:
def is_node(graph, pos, end):
    #check if node
    dxdy = [[-1,0], [1,0], [0,-1], [0,1]]
    count = 0
    for xy in dxdy:
        if pos[0]+xy[0] >= 0 and pos[0]+xy[0] < len(graph):
            if pos[1]+xy[1] >= 0 and pos[1]+xy[1] < len(graph[pos[0]+xy[0]]):
                if graph[pos[0]+xy[0],pos[1]+xy[1]] == '#':
                    count += 1
    if count < 2 or (pos[0] == end[0] and pos[1] == end[1]):
        return True
    return False

def BFS_reduce(graph, start, end):
    dxdy = [[-1,0], [1,0], [0,-1], [0,1]]
    queue = deque([start])
    dist = {start: [0, start]}
    
    nodes = {}
    nodes[start] = {}
    
    while len(queue):
        cur_pos = queue.popleft()
        cur_dist, lst_node = dist[cur_pos]
        
        nxt_dist = cur_dist + 1
        for xy in dxdy:
            nxt_pos = (cur_pos[0]+xy[0], cur_pos[1]+xy[1])
            
            if good_pos(graph, cur_pos, xy, 1) and nxt_pos not in dist.keys():
                
                queue.append(nxt_pos)
                
                if is_node(graph, nxt_pos, end):
                    nodes[lst_node][nxt_pos] = nxt_dist-dist[lst_node][0]
                    lst_node = nxt_pos
                    if lst_node not in nodes.keys():
                        nodes[lst_node] = {}
                    
                dist[nxt_pos] = [nxt_dist, lst_node]
            elif good_pos(graph, cur_pos, xy, 1) and nxt_pos in nodes.keys():
                nodes[lst_node][nxt_pos] = nxt_dist-dist[lst_node][0]
                lst_node = nxt_pos
                if lst_node not in nodes.keys():
                    nodes[lst_node] = {}
                
    return nodes
                
def clean_graph(reduced_graph, end):                
    #get final node
    for node in reduced_graph.keys():
        if end in reduced_graph[node].keys():
            extra_dist = reduced_graph[node][end]
            last_node = node
            break
            
    #replace last node with end
    for node in reduced_graph.keys():
        if last_node in reduced_graph[node].keys():
            new_dist = reduced_graph[node][last_node]+extra_dist
            reduced_graph[node][end] = new_dist
            del reduced_graph[node][last_node]
    del reduced_graph[last_node]
    
    nodes = reduced_graph.keys()
    for node in nodes:
        for key, value in reduced_graph[node].items():
            if node not in reduced_graph[key].keys():
                reduced_graph[key][node] = value
    
    return reduced_graph

def DFS_part2(graph, pos, path, end, distance):
    path.append(pos)
    
    if pos[0] == end[0] and pos[1] == end[1]:
        #print(path, np.max([distance, len(path)]))
        if len(path) > distance:
            if len(path) > 6601:
                print(len(path))
            return int(len(path))
        else:
            if distance > 6601:
                print(distance)
            return distance
        
    poss_dist = []
    
    sorted_nodes = sorted(graph[pos].items(), key=lambda x:x[1], reverse=True)
    
    for nxt, nxt_dist in sorted_nodes:#graph[pos].items():
        if nxt in path:
            continue
        poss_dist.append(DFS_part2(graph, nxt, deepcopy(path), end, distance+nxt_dist))
        
    if len(poss_dist) == 0:
        return distance
        
    return np.max(poss_dist)

def part2(data, prnt=False):
    graph = gen_graph(data)
    
    nodes = {}
    start = (0, 1)
    end = (len(graph)-1, len(graph[0])-2)
    
    nodes[start] = {}
    #print('Reduce graph...')
    reduced_graph = BFS_reduce(graph, start, end)
    #print('...Complete graph...')
    reduced_graph = clean_graph(reduced_graph, end)
    
    if prnt:
        for key in reduced_graph.keys():
            print(key, reduced_graph[key])
        print()

        for i in range(0, len(graph)):
            string = ''
            for j in range(0, len(graph[i])):
                if (i,j) in reduced_graph.keys():
                    string += 'O'
                else:
                    string += graph[i,j]
            print(string)
        print()
    
    #print('...Calculate distance')
    distance = DFS_part2(reduced_graph, start, [], end, 0)
    return distance
    

print(part2(test))
print('Part 2 result:', part2(data))

154
6602
6602


KeyboardInterrupt: 